# HW5: Building and evaluating a part-of-speech tagger

In this assignment, we are interested in what factors affect how well a trained part-of-speech tagger will do on unseen data. The homework will assess your ability to manipulate data (the output of neural language models) and your ability to discuss the results from each of the classifiers that you train. You will train four classifiers that will focus on different ways of looking at the data.

We specifically want you to assess the performance of a deep neural language model ([RoBERTa; Liu et al., 2019](https://arxiv.org/abs/1907.11692)) that is readibly available in the `huggingface` package. We would like you to compare and contrast the performance of classifiers trained on representations obtained from a lower layer of RoBERTa with classifiers trained on a higher layer from RoBERTa. This will allow you to see how different parts of a neural model's architecture do or do not encode the same kind of information.

We also want to see the effect of how close the training data is to the test data. This is like a real-world scenario, where we often train on the recent past to predict the present. So, we want to get the right part-of-speech tags for this year's abstracts (2021) either from classifiers trained on abstract part-of-speech tags from the immediately preceding year (2020) or any year prior to 2020. Think about how language and science can change while you are building these classifiers. What does it mean if 2020 and pre-2020 influence performance on tagging the 2021 dataset?

### Before you start: What are part-of-speech tags?

Part of speech tags are labels we assign to words depending on what kind of syntactic role they play in a sentence. While we have not studied part-of-speech tags yet in class, they have come up when we have talked about nouns, verbs, modifiers, etc. Building a good classifier that can do part-of-speech tagging can help us better understand things like the syntactic structure of a sentence. In order to understand the meaning of a chunk of language, we need to know what kind of "role" each word is playing.

There are lots of resources for learning lots about part of speech tags. For this assignment, we will work with the most basic of categories: "Universal" labels. These categories are designed to work for as many languages as possible. We are trying to predict the following categories in context to the best of our ability:

    VERB - verbs (all tenses and modes)
    NOUN - nouns (common and proper)
    PRON - pronouns
    ADJ - adjectives
    ADV - adverbs
    ADP - adpositions (prepositions and postpositions)
    CONJ - conjunctions
    DET - determiners
    NUM - cardinal numbers
    PRT - particles or other function words
    X - other: foreign words, typos, abbreviations
    . - punctuation

So, our classifiers will try to learn what makes something an "adjective", what makes something a "noun", and so on.

## Warning: This assignment will probably take a long time!!
## Lots of moving parts and many computations are very slow.
## Please heed the advice below:

* ### We recommend that you prototype on very small subsets of the data (e.g., `train_2020_only[0:5]` and `test_2021[0:5]`)
* ### Only once you are ready to submit your assignment and start writing up your results should you run through the whole dataset.
* ### Running RoBERTa and training your classifier can easily take half an hour or more to run depending on the efficiency of your implementation. When you have finished prototyping, expect for this to take a full 3-4 hours, just in case.

## DO NOT start until the last minute. It will only lead to avoidable suffering.

# Q1: Installing prerequisites (2 points)

In order to do this assignment, you need to install the `transformers` package from `huggingface`. Do that in the cell below.

# Q2: Imports (1 point)

Put all of the imports you will use here. Also include the neural language model and tokenizer that you will use. We will be using the RoBERTa models; for examples, refer to lecture notebooks. Keep in the `model.eval()` code below.

In [ ]:
# your imports go here



model.eval()

#Q3: Data preprocessing (6 points)

## Q3A: Loading in the three datasets (3 points)

For this assignment, we are going to use best practices in  machine learning and split our training data and our test data apart. We have two training datasets for you, which we described above. The 2020-only dataset is called `train_2020-only.json` and the pre-2020 dataset is called `train_pre-2020.json`. Each line correponds to one `json` object. Load in each of these training datasets as `train_2020_only` and `train_pre2020` respectively using our familiar friend `json.loads`, reading in the data line by line.

Our test data is stored in the file `test_2021.json`. It is structured exactly the same way as the training files, but when you load it in, name it `test_2021`.

All of the datasets are stored in the `data/` subdirectory.

## Q3B: Preview data (1 point)

Print out the first two entries of `train_2020_only`.

## Q3C: What are each of the lines? (2 points)

What kind of data structure is it? What are the elements?

<font color="red">----Your answer goes here----</font>

# Q4: Creating embeddings for each utterance (25 points) for your training data and producing four models

For hints, check out the `natural_language_inference.ipynb` functions.

Use the below function to take a single sentence and turn it into an embedding that we can use for our classifiers. This model will automatically ignore all non-initial morphemes so you do not have to worry about how RoBERTa handles word pieces.

Pay attention to the `# note` in the below for a clue to a later question.

In [ ]:
def embed_words_roberta(single_data_entry, model, tokenizer):
  words_only = [x[0] for x in single_data_entry] # note
  tokenized = tokenizer(words_only, return_tensors='pt',
                        is_split_into_words=True)
  embedded = model(**tokenized)
  embeddings = embedded['hidden_states']
  token_strings = tokenizer.convert_ids_to_tokens(tokenized['input_ids'][0].tolist())
  dimensions_to_keep = [i for i, x in enumerate(token_strings)
                        if x.startswith("Ġ") or i==1]
  subsetted_embeddings = [x[:, dimensions_to_keep].detach().numpy()
                          for x in embeddings]
  return subsetted_embeddings

## Q4A: Extract the embeddings at a specific layer (2 points)

Please print out the 7th layer from the output of `embed_words_roberta(train_2020_only[0])`.

Then print out the 3rd layer.

Remember Python indexing.

##Q4B: Complete the function `process_training_dataset` (7 points)

Your code below should be able to take a given dataset that you loaded in above and produce word embeddings for each word. Then, these word embeddings will be used to train a classifier. All you need to do is make sure your Xs and ys are shaped right and you should be good to go.

The function `process_training_dataset` critically must take in:

* A dataset (e.g., any of the above)
* A neural language model
* A tokenizer that the neural language model can work with
* A specific layer number that we subset to when building our training data

And it will return:
* A trained classifier that can assign part-of-speech tags given word embeddings

**Note**: Pay attention to your answer in the previous question so you can better extract the right word embeddings for all of your classifiers!

In [ ]:
def process_training_dataset(dataset, neural_model, neural_tokenizer, layer_number):
  # define your Xs and ys (embeddings and POS tags)
  Xs, ys = [], []
  # loop over every sentence in the dataset
    # extract POS tags for that sentence
    # combine ys with the POS tags

    # extract embeddings for that sentence

    # get embeddings at a specific layer

  Xs = np.vstack(Xs)
  classifier = LogisticRegression(max_iter=1000)
  classifier.fit(Xs, ys)
  return classifier

##Q4C: Train your model @ layer 0 on the 2020-only data (4 points)

## Q4D: Model @ layer 0, on pre-2020 data (4 points)

## Q4E: Model @ layer 10, on 2020-only data (4 points)

## Q4F: Model @ layer 10, on pre-2020 data (4 points)

# Q5: Evaluate and compare all models (18 points)

For this question, we would like you to compare each of the models to each other along several dimensions. The four models cross the layer within the model (0 or 10) and whether the model is trained on older or newer data (pre-2020 or 2020 data). In order to compare the models, you need to get each of your models to generate **predicted** labels for each of the test items. First, you will need to **construct your test dataset** and then evaluate each model along the following dimensions using the following functions:

* Precision (`sklearn.metrics.precision_score`)
* Recall (`sklearn.metrics.recall_score`)
* F1 (`sklearn.metrics.f1_score`)

Then, you will be asked to fill in the performance of each of these four models in the form of a table. You will find instances of this around some of the previous lectures (e.g., the natural language inference and evaluation lecture notebooks).

## Q5A: Test data processing (3 points)

In order for us to assess the ability for our models above to do well, we have to also process our test data. The way `scikit-learn` expects to produce predictions is very simple. When we _train_ a `LogisticRegression` model, we give the model as input some set of $X$ values (e.g., a matrix of word embeddings). The model we train tries to optimize the fit between $X$ and the $y$ values we give -- such as the labels associated with part-of-speech tags. Getting _predictions_ from our trained models is simply a matter of giving it new $X$ values -- from our test dataset.

In order for this to work, we also have to process our test data to conform to the same structure as our training data. So, for this question, we would like you to make a function `process_test_dataset` that is just like the `process_train_dataset` but there is no need to train a model at the end at all. Instead, the function _only_ needs to return `Xs` (a matrix containing word embeddings) and `ys` (part-of-speech tags). The stub of what you need to do is below.

In [ ]:
def process_test_dataset():
  Xs, ys = [], []
  # implement the test version of process_train_dataset

  return Xs, ys

## Q5B: Score all four models (12 points)

Loop through each of your four models (output of last four notebook cells), print the precision, recall, and f1 scores. 

In [ ]:
# use precision_score, recall_score, f1_score

## Q5C: Free response (3 points)

Using the outputs above, fill out a table showing performance across each of the 4 models, along all 3 measures. Describe in words how the models differ in their performance. Are there any patterns you notice that determine model performance? Were any of the results surprising to you? Why or why not? If the differences are small, can you think of a reason why we might not trust these results?

<font color="red">Your free response goes here.</font>

<font color="red">Your table goes here.</font>

# Bonus: Error analysis (6 points; 3 for code, 3 for free response)

*   Take the best-performing model
*   Construct a confusion matrix in any way that you would like, comparing the output of the best model on your test set and the true labels.
*   What categories are most confusable? What linguistic reasons might that be the case?

<font color="red">Your bonus question answer goes here.</font>